In [50]:
import os
print(os.getcwd())


/content/mlds6/MLDS6_Proyecto


In [18]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from gensim.models import Word2Vec
import numpy as np

!ls
# Load data
data = pd.read_json('../../src/bazinga/database/Clean_Headlines.json', lines=True)
headlines = data['headline']
labels = data['is_sarcastic']
data.info()

bert_embeddings.csv  bert_embeddings.npy  main.py
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   article_link  26709 non-null  object
 1   headline      26709 non-null  object
 2   is_sarcastic  26709 non-null  int64 
 3   clean_text    26709 non-null  object
dtypes: int64(1), object(3)
memory usage: 834.8+ KB


In [19]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [63]:
# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

def get_bert_embeddings(texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Use [CLS] token and move back to CPU
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings)

bert_embeddings = get_bert_embeddings(headlines.tolist())

In [64]:
# Save BERT embeddings to file
np.save('bert_embeddings.npy', bert_embeddings)
bert_df = pd.DataFrame(bert_embeddings)
bert_df.to_csv('bert_embeddings.csv', index=False)

# Load BERT embeddings from file
loaded_bert_embeddings = np.load('bert_embeddings.npy')
loaded_bert_df = pd.read_csv('bert_embeddings.csv')
loaded_bert_embeddings_from_csv = loaded_bert_df.to_numpy()

In [22]:
print(bert_embeddings)

[[-0.2787512   0.1250676  -0.43515998 ... -0.2610952   0.18953992
   0.30300787]
 [ 0.2868716   0.19836088  0.06834832 ... -0.26255423  0.18669052
   0.46911144]
 [-0.5403207  -0.23035818  0.47407225 ... -0.44920158  0.3639957
   0.01749162]
 ...
 [-0.25932276 -0.09551144 -0.61754864 ... -0.05595682  0.46902227
   0.20649998]
 [-0.09609814  0.23278977 -0.24334513 ... -0.4787058  -0.01567005
   0.11095161]
 [-0.25170836  0.01995322  0.24349986 ... -0.31329057 -0.02248631
   0.3239032 ]]


In [65]:
# Tokenize the headlines
tokenized_headlines = [headline.split() for headline in headlines]

# Train Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_headlines, vector_size=300, window=5, min_count=1, workers=4)

# Generate Word2Vec embeddings
def get_w2v_embeddings(texts, model):
    embeddings = []
    for text in texts:
        words = text.split()
        word_vectors = [model.wv[word] for word in words if word in model.wv]
        if word_vectors:
            embeddings.append(np.mean(word_vectors, axis=0))
        else:
            embeddings.append(np.zeros(300))  # Handle empty cases
    return np.array(embeddings)

w2v_embeddings = get_w2v_embeddings(headlines, w2v_model)

In [66]:
# Save Word2Vec embeddings
np.save('w2v_embeddings.npy', w2v_embeddings)
w2v_df = pd.DataFrame(w2v_embeddings)

# Save to CSV
w2v_df.to_csv('w2v_embeddings.csv', index=False)
loaded_w2v_embeddings = np.load('w2v_embeddings.npy')
loaded_w2v_df = pd.read_csv('w2v_embeddings.csv')
loaded_w2v_embeddings = loaded_w2v_df.to_numpy()

In [26]:
print(w2v_embeddings)

[[-0.0041957   0.22404993  0.09574366 ... -0.13403803  0.20319869
  -0.06994065]
 [-0.01266595  0.39859638  0.17234276 ... -0.21259151  0.34799477
  -0.12914315]
 [-0.01191726  0.39524963  0.17347732 ... -0.18527654  0.3363047
  -0.13570245]
 ...
 [-0.01523993  0.49155304  0.20881777 ... -0.27298486  0.4282649
  -0.15189879]
 [-0.00379999  0.09126084  0.03992998 ... -0.05554602  0.0829154
  -0.02970187]
 [-0.00941545  0.31437373  0.13352586 ... -0.1776877   0.2778742
  -0.09951697]]
